In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Estableciendo directorio
tar = os.chdir(r"undisclosed")

# Leyendo base
df_contacts = pd.read_csv("df_contacts_cleaned.csv")

In [ ]:
def cluster_text(column, column_str):

    #Devuelve clusters a partir de columnas de texto.
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(column)

    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    Sum_of_squared_distances = []
    K = range(2,10)
    for k in K:
       km = KMeans(n_clusters=k, max_iter=200, n_init=10)
       km = km.fit(X)
       Sum_of_squared_distances.append(km.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    print('How many clusters do you want to use?')
    true_k = int(input())
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
    model.fit(X)

    labels=model.labels_
    df_cluster=pd.DataFrame(list(zip(column,labels)),columns=['title',f'cluster_{column_str}'])
    #print(clusters.sort_values(by=['cluster']))
        
    return df_cluster

In [ ]:
column_list_cluster = ['profesion_procesada_corregida']

for column in column_list_cluster:
    df_contacts_column = pd.DataFrame()
    df_contacts_column[['email', column]] = df_contacts[['email', column]].dropna().reset_index(drop = True)
    df_contacts_cluster_column = cluster_text(df_contacts_column[column], column)
    df_contacts_column_res_index = df_contacts_column.reset_index(drop = True)
    df_contacts_column = pd.merge(
        df_contacts_column_res_index, 
        df_contacts_cluster_column, 
        left_index=True, 
        right_index=True
    )
    df_contacts_column = df_contacts_columnds.drop()
    df_contacts = pd.merge(df_contacts, df_contacts_column, how = 'left', on = 'email')

df_contacts.cluster_profesion.value_counts()

In [ ]:
df_contacts.to_csv('df_contacts_clusterized.csv')
df_contacts.to_excel('df_contacts_clusterized.xlsx')